# Image Captioning — Dataset y DataLoader

En este notebook se define el Dataset personalizado y el DataLoader
que se utilizarán para entrenar el modelo de Image Captioning.

## Por qué usar Dataset y DataLoader

En PyTorch, el uso de Dataset y DataLoader permite:
- Separar la lógica de carga de datos del modelo
- Trabajar con batches de forma eficiente
- Mantener el código modular y reutilizable

Este enfoque facilita el entrenamiento y la depuración del modelo.

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

In [24]:
captions_df = pd.read_csv(
    "../data/captions_processed.txt",
    names=["image", "caption"]
)

## Preprocesamiento de imágenes

Las imágenes se redimensionan y normalizan para ser compatibles
con la CNN preentrenada utilizada como encoder.

In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

## Dataset personalizado

El Dataset devuelve:
- una imagen preprocesada
- la caption correspondiente en forma de secuencia numérica

In [26]:
class CaptionDataset(Dataset):
    def __init__(self, captions_df, images_path, word2idx, max_length, transform=None):
        self.captions_df = captions_df
        self.images_path = images_path
        self.word2idx = word2idx
        self.max_length = max_length
        self.transform = transform

    def __len__(self):
        return len(self.captions_df)

    def __getitem__(self, idx):
        row = self.captions_df.iloc[idx]
        image_path = os.path.join(self.images_path, row["image"])
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        caption = row["caption"].split()
        caption_idx = [self.word2idx[word] for word in caption]

        # padding
        if len(caption_idx) < self.max_length:
            caption_idx += [0] * (self.max_length - len(caption_idx))
        else:
            caption_idx = caption_idx[:self.max_length]

        caption_tensor = torch.tensor(caption_idx)

        return image, caption_tensor

In [27]:
import pickle

with open("../data/text_artifacts.pkl", "rb") as f:
    artifacts = pickle.load(f)

word2idx = artifacts["word2idx"]
idx2word = artifacts["idx2word"]
max_length = artifacts["max_length"]

print("Artefactos cargados:")
print("Vocab size:", len(word2idx))
print("Max length:", max_length)

Artefactos cargados:
Vocab size: 2960
Max length: 20


In [28]:
dataset = CaptionDataset(
    captions_df=captions_df,
    images_path="../data/images_subset",
    word2idx= word2idx,
    max_length= max_length,
    transform=transform
)

dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True
)

## Comprobación del DataLoader

Se comprueba que el DataLoader devuelve batches con
la forma esperada antes de entrenar el modelo.

In [29]:
images, captions = next(iter(dataloader))
print(images.shape)
print(captions.shape)

torch.Size([32, 3, 224, 224])
torch.Size([32, 20])
